In [ ]:
import torch
from torch_geometric.data import Data, Dataset
import networkx as nx
from networkx.algorithms.centrality import betweenness_centrality

from Datasets.synthetics import BA_2grid, BA_2grid_house, ProbingDataset, BA_2grid_to_test
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader

import torch_geometric.utils as pyg_utils
import networkx as nx
from networkx.algorithms.centrality import betweenness_centrality

import pickle as pkl
from torch_geometric.utils import from_networkx
import random

from models.models_BA_2grid import Cheb_framework as framework
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import torch.optim as optim

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def set_seed(seed):
    if seed == -1:
        seed = random.randint(0, 1000)
    # Pandas also uses np random state by default
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    # if you are using GPU
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
set_seed(43)

# Compute Graph-Features Dataset

The features for BA-2grid-house and ER-nb_stars2 datasets:
- num_nodes
- num_edges
- density
- average_shortest_path_length
- transitivity
- average_clustering

In [ ]:
probing_dataset = ProbingDataset()

In [ ]:
probing_dataset

In [ ]:
gnn_dataset_to_test = BA_2grid_to_test()

In [ ]:
idx = torch.arange(len(gnn_dataset_to_test))
train_idx, test_idx = train_test_split(idx, train_size=0.8,random_state=10)

train_loader = DataLoader(gnn_dataset_to_test[train_idx],batch_size=256)
test_loader = DataLoader(gnn_dataset_to_test[test_idx],batch_size=256)

In [ ]:
# Assuming you have a list of graphs in your dataset
idx = torch.arange(len(probing_dataset))
train_idx, test_idx = train_test_split(idx, train_size=0.8,random_state=10)

probe_train_loader = DataLoader(probing_dataset[train_idx],batch_size=256)
probe_test_loader = DataLoader(probing_dataset[test_idx],batch_size=256)

# Load Model to Probe

In [ ]:
MODEL = "Chab"
DATASET = "BA_2grid"
dataset = BA_2grid()
gnn = framework(dataset,device="cpu")

In [ ]:
# train the model
gnn.train()

# Train probe

In [ ]:
class Probe(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Probe, self).__init__()
        self.fc = torch.nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.fc(x.float())
        return out

In [ ]:
def train_probe(probe, gnn, layer_idx):
    gnn.model.eval()
    probe.train()
    
    total_loss = 0
    loss_fn = torch.nn.MSELoss()
    optimizer = optim.Adam(probe.parameters(), lr=0.001)

    for graph in probe_train_loader:

        optimizer.zero_grad()
        output = gnn.model(graph.x, graph.edge_index, graph.batch)
        output = torch.sum(output, dim=1)
        labels = graph.y.reshape(int(len(graph.y)/6), 6)
        padded_output = torch.nn.functional.pad(output, (0, 256 - int(len(graph.y)/6)))
        output = probe(padded_output)
        
        
        loss = loss_fn(output, labels)
        loss.backward()

        optimizer.step()
        
        total_loss += float(loss) * graph.num_graphs
    return total_loss / len(probe_train_loader.dataset)

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_probe(probe, test_loader):
    probe.eval()

    mse = 0
    r2 = 0
    total = 0
    loss_fn = torch.nn.MSELoss()
    
    for graph in test_loader:

        output = gnn.model(graph.x, graph.edge_index, graph.batch)
        output = torch.sum(output, dim=1)
        y_true = graph.y.reshape(int(len(graph.y)/6), 6)
        padded_output = torch.nn.functional.pad(output, (0, 256 - int(len(graph.y)/6)))
        y_pred = probe(padded_output)

        mse += loss_fn(y_pred, y_true)
        # r2 += r2_score(y_true, y_pred)
        total += 1
    
    return {
        "MSE": mse/total
        # "R2 Score": r2/total
    }

In [ ]:
def iterate_to_train(gnn, layer_idx, num_epochs=10):
    probe = Probe(256, 6)
    losses = []
    test_losses = []
    for epoch in range(num_epochs):
        loss = train_probe(probe, gnn, layer_idx)
        losses.append(loss)
        result = evaluate_probe(probe, probe_test_loader)
        test_acc = result['MSE']
        test_losses.append(test_acc)
        print(f'Epoch: {epoch:03d}, '
              f'Test MSE: {test_acc:.3f}',
              f'Train MSE: {loss:.3f}')
        # print(f'Epoch: {epoch:03d}, '
        #       f'Loss: {loss:.3f}, ')
    return probe, losses, test_losses

In [ ]:
probe, losses, test_acc = iterate_to_train(gnn, 0, num_epochs=100)

In [ ]:
import matplotlib.pyplot as plt
epochs = [i for i in range(100)]
plt.plot(epochs, losses)

In [ ]:
gnn.model.eval()
loss_fn = torch.nn.MSELoss()
mse = 0
total = 0 
r2_scores = []
for graph in train_loader:
    output = gnn.model(graph.x, graph.edge_index, graph.batch)
    output = torch.sum(output, dim=1)
    y_true = graph.y.reshape(int(len(graph.y)/6), 6)
    padded_output = torch.nn.functional.pad(output, (0, 256 - int(len(graph.y)/6)))
    y_pred = probe(padded_output)
    
    mse += loss_fn(y_pred, y_true)
    total += 1

    # Calculate R2 score per property
    r2_per_property = []
    for i in range(len(y_true)):
        r2 = r2_score(y_true[i], y_pred.detach().numpy())
        r2_per_property.append(r2)
    
    r2_scores.append(r2_per_property)

print(mse/total)
print(r2_scores)

In [ ]:
gnn.model.eval()
loss_fn = torch.nn.MSELoss()
mse = 0
total = 0 
r2_scores = []
for graph in train_loader:
    output = gnn.model(graph.x, graph.edge_index, graph.batch)
    output = torch.sum(output, dim=1)
    y_true = graph.y.reshape(int(len(graph.y)/6), 6)
    padded_output = torch.nn.functional.pad(output, (0, 256 - int(len(graph.y)/6)))
    y_pred = probe(padded_output)
    
    mse += loss_fn(y_pred, y_true)
    total += 1

    # Calculate R2 score per property
    r2_per_property = []
    for i in range(len(y_true)):
        r2 = r2_score(y_true[i], y_pred.detach().numpy())
        r2_per_property.append(r2)
    
    r2_per_property = np.mean(r2_per_property)
    r2_scores.append(r2_per_property)

# r2_scores = np.mean(r2_scores, axis=0)
print(mse/total)
print(np.mean(r2_scores))
print(r2_scores)